In [1]:
import pandas as pd
import os
import numpy as np
import glob

import soundfile as sf

In [2]:
data_path = './data/wav/'
sr = 48000

sliced_wav_path = './data/sliced_wav/'

In [3]:
# get pation id list from wav folder names
def get_patient_id_list(wav_folder):
    patient_id_list = []
    for folder in os.listdir(wav_folder):
        patient_id_list.append(folder)
    return patient_id_list

In [4]:
patient_list = get_patient_id_list(data_path)

In [5]:
patient_list

['00001037', '00001371', '00001406', '00001576', '00001590']

In [6]:
# get all the wav files in the folder using patient id
def get_wav_files(patient_id):
    wav_file_list = []
    for wav_file in glob.glob(data_path + patient_id + '/*.wav'):
        wav_file_list.append(wav_file)
    return wav_file_list

In [7]:
wav_file_list = get_wav_files(patient_list[0])

In [8]:
def get_wav_file_name(wav_file_path):   
    return wav_file_path.split('\\')[-1].split('.')[0]

In [9]:
# segment the wav file 20 sec frame and 10 sec overlap and save it in the sliced_wav folder
# if the final frame of the file is less than 20 sec, just save it as it is
# make created files list

def segment_wav_file(sliced_wav_path, patient_id, frame_size, overlap):
    wav_file_list = get_wav_files(patient_id)
    save_dir = sliced_wav_path + patient_id
    sliced_wav_file_list = []

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    for wav_file in wav_file_list:
        wav_file_name = get_wav_file_name(wav_file)
        y, sr = sf.read(wav_file)
        file_number = 0
        for i in range(0, len(y), overlap*sr):
            if len(y[i:i+frame_size*sr]) == frame_size*sr:
                sf.write(save_dir + '/' + str(wav_file_name) + '_' + str(file_number).zfill(3) + '.wav', y[i:i+frame_size*sr], sr)
                sliced_wav_file_list.append(save_dir + '/' + str(wav_file_name) + '_' + str(file_number).zfill(3) + '.wav')
                file_number += 1
            else:
                sf.write(save_dir + '/' + str(wav_file_name) + '_' + str(file_number).zfill(3) + '.wav', y[i:], sr)
                sliced_wav_file_list.append(save_dir + '/' + str(wav_file_name) + '_' + str(file_number).zfill(3) + '.wav')
                break
    return sliced_wav_file_list

In [10]:
test_list = segment_wav_file(sliced_wav_path, patient_list[0], 20, 10)